In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests

In [76]:
pokemon = pd.DataFrame(columns = ["number", "name", "type", "ability", "hidden_ability", 
                                 "gender_ratio", "hatch_cycle", "Base_Experience_Yield", "height", "weight",
                                 "leveling_rate", "color", "base_friendship", "category"], index = range(1, 1026))
pokemon

,number,name,type,ability,hidden_ability,gender_ratio,hatch_cycle,Base_Experience_Yield,height,weight,leveling_rate,color,base_friendship,category
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
pokemon = pd.read_csv("/home/user/Desktop/Data_analysis_projects/Pokemon/data.csv")

In [201]:
# Creating a function for scraping pokemon data

def scrap_pokemon_details(pokemon_name):
    """
    scrape details of pokemon from bulbapedia

    Args :
        pokemon_name(str) name of the pokemon, eg : Greninja 

    Returns:
        A dictionary will all the details 
    """
    
    try: 
        # Give URL 
        url = f"https://bulbapedia.bulbagarden.net/wiki/{pokemon_name}_(Pok%C3%A9mon)"
        
        # fetching the webpage 
        response = requests.get(url)
        response.raise_for_status()
        text = response.text
        
        # Creating a BeautifulSoup instance 
        soup = BeautifulSoup(text, "lxml")
        
        # Overall data 
        data = soup.find_all("td", class_ = "roundy")
        
        if len(data) < 40 :
            print(f"Don't have enought data for {pokemon_data}")
            return None
        
        # name, number, category
        basic = data[0]
        name = basic.find(["big", "b"]).text.strip()
        category = basic.find(["span"]).text.strip()
        number = soup.find(["th","span", "big"], class_ = "roundy").text.replace("\n", "").strip()
        
        # Type 
        type_data = data[1]
        type_data_2 =  type_data.find_all(["span", "a"])
        type_ = type_data_2[2].text.strip(), type_data_2[4].text.strip()
        
        # ability and hidden ability
        ability_data = data[2]
        ability_data2 = ability_data.find_all(["a", "span"])
        clear_ability = []
        junk_text = ["Cacophony", "Abilities", "Gen IV+", "Battle Bond", "Battle Bond Greninja", "Gen VI+", "Gen VII+", "Ability"]

        for i in ability_data2:
            text = i.text.strip()
            if text and text not in junk_text:
                clear_ability.append(text)
        
        if len(clear_ability) == 6:
            ability = clear_ability[0], clear_ability[2]
            hidden_ability = clear_ability[4]
        elif len(clear_ability) == 4:
            ability = clear_ability[0]
            hidden_ability = clear_ability[2]
        elif len(clear_ability) == 2:
            ability = clear_ability[0]
            hidden_ability = None
        else:
            ability = None
            hidden_ability = None
            
        # gender ratio
        gender_data = data[3]
        gender_data_2 = gender_data.find_all(["a"])
        gender_ratio = gender_data_2[1].text.strip()
        
        # Catch rate 
        catch_rate = data[4].find("small").text.strip()
        
        # egg group and hatch cycle 
        egg_group_data = data[5]
        egg_group = egg_group_data.find("table", class_ = "roundy").text.strip()
        hatch_cycle_data = data[5].find_all("table", class_ = "roundy")
        hatch_cycle = hatch_cycle_data[1].text.strip().replace("\xa0", "")
        
        # Height and Weight 
        height_data = data[6].find_all("tr")
        height_data_2 = height_data[0].find_all("td")
        height = height_data_2[1].text.replace("\n", "").strip()
        weight_data = data[7].find_all("tr")
        weight_data_2 = weight_data[0].find_all("td")
        weight = weight_data_2[1].text.replace("\n", "").strip()
        
        # Base Experience Yield 
        bey_data = data[9].find("tr")
        bey_data_2 = bey_data.find_all("td")
        bey = bey_data_2[2].text.replace("V+", "").replace("\n", "").strip()
        
        # Other details 
        leveling_rate = data[10].tr.text.strip()
        color = data[38].td.text.strip().replace("Other forms may have other colors.", "")
        base_friendship = data[39].tr.text.strip()
        
        return {"number" : number, "name" : name, "type" : type_, "ability" : ability, "hidden_ability" : hidden_ability, 
                 "gender_ratio" : gender_ratio, "hatch_cycle" : hatch_cycle, "Base_Experience_Yield" : bey, "leveling_rate" : leveling_rate,
                 "color" : color, "base_friendship" : base_friendship, "height" : height, "weight" : weight, "category" : category}
            
    except requests.RequestException as e:
        print(f"Failed to fetch data for {pokemon_name}: {e}")
        return None
    except (AttributeError, IndexError) as e:
        print(f"Failed to parse data for {pokemon_name}: {e}")
        return None

            

In [78]:
# Bulbasaur evolution line
bulbasaur_data = scrap_pokemon_details("Bulbasaur")
pokemon.loc[1] = bulbasaur_data
ivysaur_data = scrap_pokemon_details("Ivysaur")
pokemon.loc[2] = ivysaur_data
venusaur_data = scrap_pokemon_details("Venusaur")
pokemon.loc[3] = venusaur_data

In [79]:
# Charmander line 
charmander_data = scrap_pokemon_details("Charmander")
pokemon.loc[4] = charmander_data
charmeleon_data = scrap_pokemon_details("Charmeleon")
pokemon.loc[5] = charmeleon_data
charizard_data = scrap_pokemon_details("Charizard")
pokemon.loc[6] = charizard_data

In [80]:
# Squirtle line 
squirtle_data = scrap_pokemon_details("Squirtle")
pokemon.loc[7] = squirtle_data
wartortle_data = scrap_pokemon_details("Wartortle")
pokemon.loc[8] = wartortle_data
blastoise_data = scrap_pokemon_details("Blastoise")
pokemon.loc[9] = blastoise_data

In [81]:
# Caterpie line 
caterpie_data = scrap_pokemon_details("Caterpie")
pokemon.loc[10] = caterpie_data
metapod_data = scrap_pokemon_details("Metapod")
pokemon.loc[11] = metapod_data
butterfree_data = scrap_pokemon_details("Butterfree")
pokemon.loc[12] = butterfree_data

In [82]:
# Weedle line 
weedle_data = scrap_pokemon_details("Weedle")
pokemon.loc[13] = weedle_data
kakuna_data = scrap_pokemon_details("Kakuna")
pokemon.loc[14] = kakuna_data
beedrill_data = scrap_pokemon_details("Beedrill")
pokemon.loc[15] = beedrill_data

In [83]:
#pidgey line
pidgey_data = scrap_pokemon_details("Pidgey")
pokemon.loc[16] = pidgey_data
pidgeotto_data = scrap_pokemon_details("Pidgeotto")
pokemon.loc[17] = pidgeotto_data
pidgeot_data = scrap_pokemon_details("Pidgeot")
pidgeot_data["ability"] = "Keen Eye", "Tangled Feet"
pidgeot_data["hidden_ability"] = "Big Pecks"
pokemon.loc[18] = pidgeot_data

In [84]:
# Rattata line
rattata_data = scrap_pokemon_details("Rattata")
rattata_data["ability"] = "Run Away", "Guts"
rattata_data["hidden_ability"] = "Hustle"
pokemon.loc[19] = rattata_data
raticate_data = scrap_pokemon_details("Raticate")
raticate_data["ability"] = "Run Away", "Guts"
raticate_data["hidden_ability"] = "Hustle"
pokemon.loc[20] = raticate_data

In [88]:
#Spearow line
spearow_data = scrap_pokemon_details("Spearow")
pokemon.loc[21] = spearow_data
fearow_data = scrap_pokemon_details("Fearow")
pokemon.loc[22] = fearow_data

In [90]:
# Ekans line
ekans_data = scrap_pokemon_details("Ekans")
pokemon.loc[23] = ekans_data
arbok_data = scrap_pokemon_details("Arbok")
pokemon.loc[24] = arbok_data

In [94]:
#Pikachu line
pikachu_data = scrap_pokemon_details("Pikachu")
pokemon.loc[25] = pikachu_data
raichu_data = scrap_pokemon_details("Raichu")
raichu_data["ability"] = "Static"
pokemon.loc[26] = raichu_data


In [98]:
#Sandshrew line
sandshrew_data = scrap_pokemon_details("Sandshrew")
sandshrew_data["ability"] = "Sand Veil"
sandshrew_data["hidden_ability"] = "Sand Rush"
pokemon.loc[27] = sandshrew_data
sandslash_data = scrap_pokemon_details("Sandslash")
sandslash_data["ability"] = "Sand Veil"
sandslash_data["hidden_ability"] = "Sand Rush"
pokemon.loc[28] = sandslash_data

In [107]:
# nidoran female line
nidoranf_data = scrap_pokemon_details("Nidoran♀")
pokemon.loc[29] = nidoranf_data
nidorina_data = scrap_pokemon_details("Nidorina")
pokemon.loc[30] = nidorina_data
nidoqueen_data = scrap_pokemon_details("Nidoqueen")
pokemon.loc[31] = nidoqueen_data

In [111]:
# Nidoran male line
nidoranm_data = scrap_pokemon_details("Nidoran♂")
pokemon.loc[32] = nidoranm_data
nidorino_data = scrap_pokemon_details("Nidorino")
pokemon.loc[33] = nidorino_data
nidoking_data = scrap_pokemon_details("Nidoking")
pokemon.loc[34] = nidoking_data

In [114]:
#Clefairy line 
clefairy_data = scrap_pokemon_details("Clefairy")
pokemon.loc[35] = clefairy_data
clefable_data = scrap_pokemon_details("Clefable")
pokemon.loc[36] = clefable_data

In [118]:
# Vulpix line
vulpix_data = scrap_pokemon_details("Vulpix")
vulpix_data["ability"] = "Flash Fire"
vulpix_data["hidden_ability"] = "Drought"
pokemon.loc[37] = vulpix_data
ninetails_data = scrap_pokemon_details("Ninetales")
ninetails_data["ability"] = "Flash Fire"
ninetails_data["hidden_ability"] = "Drought"
pokemon.loc[38] = ninetails_data

In [127]:
# Jigglypuff line
jigglypuff_data = scrap_pokemon_details("Jigglypuff")
pokemon.loc[39] = jigglypuff_data
wigglytuff_data = scrap_pokemon_details("Wigglytuff")
pokemon.loc[40] = wigglytuff_data

In [129]:
#Zubat line
zubat_data = scrap_pokemon_details("Zubat")
pokemon.loc[41] = zubat_data
golbat_data = scrap_pokemon_details("Golbat")
pokemon.loc[42] = golbat_data

In [132]:
#Oddish line
oddish_data = scrap_pokemon_details("Oddish")
pokemon.loc[43] = oddish_data
gloom_data = scrap_pokemon_details("Gloom")
pokemon.loc[44] = gloom_data
vileplume_data = scrap_pokemon_details("Vileplume")
pokemon.loc[45] = vileplume_data

In [134]:
# paras line
paras_data = scrap_pokemon_details("Paras")
pokemon.loc[46] = paras_data
parasect_data = scrap_pokemon_details("Parasect")
pokemon.loc[47] = parasect_data

In [137]:
# Venonat line
venonat_data = scrap_pokemon_details("Venonat")
pokemon.loc[48] = venonat_data
venomoth_data = scrap_pokemon_details("Venomoth")
pokemon.loc[49] = venomoth_data

In [141]:
# diglett line
diglett_data = scrap_pokemon_details("Diglett")
diglett_data["ability"] = "Sand Veil", "Arena Trap"
diglett_data["hidden_ability"] = "Sand Force"
pokemon.loc[50] = diglett_data
dugtrio_data = scrap_pokemon_details("Dugtrio")
dugtrio_data["ability"] = "Sand Veil", "Arena Trap"
dugtrio_data["hidden_ability"] = "Sand Force"
pokemon.loc[51] = dugtrio_data

In [145]:
# meowth line
meowth_data = scrap_pokemon_details("Meowth")
meowth_data["ability"] = "Pickup", "Technician"
meowth_data["hidden_ability"] = "Unnerve"
pokemon.loc[52] = meowth_data
persian_data = scrap_pokemon_details("Persian")
persian_data["ability"] = "Limber", "Technician"
persian_data["hidden_ability"] = "Unnerve"
pokemon.loc[53] = persian_data 

In [147]:
# Psyduck line
psyduck_data = scrap_pokemon_details("Psyduck")
pokemon.loc[54] = psyduck_data
golduck_data = scrap_pokemon_details("Golduck")
pokemon.loc[55] = golduck_data

In [149]:
# Mankey line 
mankey_data = scrap_pokemon_details("Mankey")
pokemon.loc[56] = mankey_data
primeape_data = scrap_pokemon_details("Primeape")
pokemon.loc[57] = primeape_data

In [156]:
# Growlithe line
growlithe_data = scrap_pokemon_details("Growlithe")
growlithe_data["ability"] = "Intimidate", "Flash Fire"
growlithe_data["hidden_ability"] = "Justified"
pokemon.loc[58] = growlithe_data
arcanine_data = scrap_pokemon_details("Arcanine")
arcanine_data["ability"] = "Intimidate", "Flash Fire"
arcanine_data["hidden_ability"] = "Justified"
pokemon.loc[59] = arcanine_data

In [161]:
# poliwag line
poliwag_data = scrap_pokemon_details("Poliwag")
pokemon.loc[60] = poliwag_data
poliwhirl_data = scrap_pokemon_details("Poliwhirl")
pokemon.loc[61] = poliwhirl_data
poliwrath_data = scrap_pokemon_details("Poliwrath")
pokemon.loc[62] = poliwrath_data

In [166]:
# Abra line 
abra_data = scrap_pokemon_details("Abra")
pokemon.loc[63] = abra_data
kadabra_data = scrap_pokemon_details("Kadabra")
pokemon.loc[64] = kadabra_data
alakazam_data = scrap_pokemon_details("Alakazam")
alakazam_data["ability"] = "Synchronize", "Inner Focus"
alakazam_data["hidden_ability"] = "Magic Guard"
pokemon.loc[65] = alakazam_data

In [168]:
# Machop line
machop_data = scrap_pokemon_details("Machop")
pokemon.loc[66] = machop_data
machoke_data = scrap_pokemon_details("Machoke")
pokemon.loc[67] = machoke_data
machamp_data = scrap_pokemon_details("Machamp")
pokemon.loc[68] = machamp_data

In [170]:
# bellsprout line 
bellsprout_data = scrap_pokemon_details("Bellsprout")
pokemon.loc[69] = bellsprout_data
weepinbell_data = scrap_pokemon_details("Weepinbell")
pokemon.loc[70] = weepinbell_data
victreebel_data = scrap_pokemon_details("Victreebel")
pokemon.loc[71] = victreebel_data

In [172]:
# tentacool line
tentacool_data = scrap_pokemon_details("Tentacool")
pokemon.loc[72] = tentacool_data
tentacruel_data = scrap_pokemon_details("Tentacruel")
pokemon.loc[73] = tentacruel_data

In [174]:
# geodude line 
geodude_data = scrap_pokemon_details("Geodude")
geodude_data["ability"] = "Rock Head", "Sturdy"
geodude_data["hidden_ability"] = "Sand Veil"
pokemon.loc[74] = geodude_data
graveler_data = scrap_pokemon_details("Graveler")
graveler_data["ability"] = "Rock Head", "Sturdy"
graveler_data["hidden_ability"] = "Sand Veil"
pokemon.loc[75] = graveler_data
golem_data = scrap_pokemon_details("Golem")
golem_data["ability"] = "Rock Head", "Sturdy"
golem_data["hidden_ability"] = "Sand Veil"
pokemon.loc[76] = golem_data

In [180]:
# ponyta line 
ponyta_data = scrap_pokemon_details("Ponyta")
ponyta_data["ability"] = "Run Away", "Flash Fire"
ponyta_data["hidden_ability"] = "Flame Body"
ponyta_data["category"] = "Fire Horse And Unique Horn Pokémon"
pokemon.loc[77] = ponyta_data
rapidash_data = scrap_pokemon_details("Rapidash")
rapidash_data["ability"] = "Run Away", "Flash Fire"
rapidash_data["hidden_ability"] = "Flame Body"
rapidash_data["category"] = "Fire Horse And Unique Horn Pokémon"
pokemon.loc[78] = rapidash_data

In [182]:
# Slowpoke line 
slowpoke_data = scrap_pokemon_details("Slowpoke")
slowpoke_data["ability"] = "Oblivious", "Own Tempo"
slowpoke_data["hidden_ability"] = "Regenerator"
pokemon.loc[79] = slowpoke_data
slowbro_data = scrap_pokemon_details("Slowbro")
slowbro_data["ability"] = "Oblivious", "Own Tempo"
slowbro_data["hidden_ability"] = "Regenerator"
pokemon.loc[80] = slowbro_data

In [184]:
# magnemite line
magnemite_data = scrap_pokemon_details("Magnemite")
pokemon.loc[81] = magnemite_data
magneton_data = scrap_pokemon_details("Magneton")
pokemon.loc[82] = magneton_data

In [186]:
# Farfetch'd
farfetch_data = scrap_pokemon_details("Farfetch'd")
pokemon.loc[83] = farfetch_data

In [189]:
# doduo line
doduo_data = scrap_pokemon_details("Doduo")
pokemon.loc[84] = doduo_data
dodrio_data = scrap_pokemon_details("Dodrio")
pokemon.loc[85] = dodrio_data

In [191]:
# Seel line 
seel_data = scrap_pokemon_details("Seel")
pokemon.loc[86] = seel_data
dewgong_data = scrap_pokemon_details("Dewgong")
pokemon.loc[87] = dewgong_data

In [192]:
# grimer line 
grimer_data = scrap_pokemon_details("Grimer")
grimer_data["ability"] = "Stench", "Sticky Hold"
grimer_data["hidden_ability"] = "Poison Touch"
pokemon.loc[88] = grimer_data
muk_data = scrap_pokemon_details("Muk")
muk_data["ability"] = "Stench", "Sticky Hold"
muk_data["hidden_ability"] = "Poison Touch"
pokemon.loc[89] = muk_data

In [194]:
# Shellder line 
shellder_data = scrap_pokemon_details("Shellder")
pokemon.loc[90] = shellder_data
cloyster_data = scrap_pokemon_details("Cloyster")
pokemon.loc[91] = cloyster_data

In [202]:
# Gastly line
gastly_data = scrap_pokemon_details("Gastly")
pokemon.loc[92] = gastly_data
haunter_data = scrap_pokemon_details("Haunter")
pokemon.loc[93] = haunter_data
gengar_data = scrap_pokemon_details("Gengar")
gengar_data["ability"] = "Cursed Body"
pokemon.loc[94] = gengar_data

In [204]:
# onix line 
onix_data = scrap_pokemon_details("Onix")
pokemon.loc[95] = onix_data

In [205]:
# drowzee line 
drowzee_data = scrap_pokemon_details("Drowzee")
pokemon.loc[96] = drowzee_data
hypno_data = scrap_pokemon_details("Hypno")
pokemon.loc[97] = hypno_data

In [206]:
# Krabby line 
krabby_data = scrap_pokemon_details("Krabby")
pokemon.loc[98] = krabby_data
kingler_data = scrap_pokemon_details("Kingler")
pokemon.loc[99] = kingler_data

In [207]:
# voltorb line 
voltorb_data = scrap_pokemon_details("Voltorb")
pokemon.loc[100] = voltorb_data
electrode_data = scrap_pokemon_details("Electrode")
pokemon.loc[101] = electrode_data

In [208]:
pokemon.to_csv("/home/user/Desktop/Data_analysis_projects/Pokemon/data.csv")
pokemon[83:101]

,number,name,type,ability,hidden_ability,gender_ratio,hatch_cycle,Base_Experience_Yield,height,weight,leveling_rate,color,base_friendship,category
84,#0084,Doduo,"(Normal, Flying)","(Run Away, Early Bird)",Tangled Feet,"50% male, 50% female",20cycles,62,1.4 m,39.2 kg,Medium Fast,Brown,70,Twin Bird Pokémon
85,#0085,Dodrio,"(Normal, Flying)","(Run Away, Early Bird)",Tangled Feet,"50% male, 50% female",20cycles,165,1.8 m,85.2 kg,Medium Fast,Brown,70,Triple Bird Pokémon
86,#0086,Seel,"(Water, Unknown)","(Thick Fat, Hydration)",Ice Body,"50% male, 50% female",20cycles,65,1.1 m,90.0 kg,Medium Fast,White,70,Sea Lion Pokémon
87,#0087,Dewgong,"(Water, Ice)","(Thick Fat, Hydration)",Ice Body,"50% male, 50% female",20cycles,166,1.7 m,120.0 kg,Medium Fast,White,70,Sea Lion Pokémon
88,#0088,Grimer,"(Poison, Unknown)","(Stench, Sticky Hold)",Poison Touch,"50% male, 50% female",20cycles,65,0.9 m,30.0 kg,Medium Fast,Purple,70,Sludge Pokémon
89,#0089,Muk,"(Poison, Unknown)","(Stench, Sticky Hold)",Poison Touch,"50% male, 50% female",20cycles,175,1.2 m,30.0 kg,Medium Fast,Purple,70,Sludge Pokémon
90,#0090,Shellder,"(Water, Unknown)","(Shell Armor, Skill Link)",Overcoat,"50% male, 50% female",20cycles,61,0.3 m,4.0 kg,Slow,Purple,70,Bivalve Pokémon
91,#0091,Cloyster,"(Water, Ice)","(Shell Armor, Skill Link)",Overcoat,"50% male, 50% female",20cycles,184,1.5 m,132.5 kg,Slow,Purple,70,Bivalve Pokémon
92,#0092,Gastly,"(Ghost, Poison)",Levitate,None,"50% male, 50% female",20cycles,62,1.3 m,0.1 kg,Medium Slow,Purple,70,Gas Pokémon
93,#0093,Haunter,"(Ghost, Poison)",Levitate,None,"50% male, 50% female",20cycles,142,1.6 m,0.1 kg,Medium Slow,Purple,70,Gas Pokémon


In [122]:
text = requests.get("https://bulbapedia.bulbagarden.net/wiki/Wigglytuff_(Pok%C3%A9mon)").text
soup = BeautifulSoup(text, "lxml")
data = soup.find_all("td", class_ = "roundy")

In [126]:
# ability and hidden ability
ability_data = data[2]
ability_data2 = ability_data.find_all(["a", "span"])
clear_ability = []
junk_text = ["Cacophony", "Abilities", "Gen IV+", "Battle Bond", "Battle Bond Greninja", "Gen VI+"]

for i in ability_data2:
    text = i.text.strip()
    if text and text not in junk_text:
        clear_ability.append(text)
        
if len(clear_ability) > 5:
    ability = clear_ability[0], clear_ability[2]
    hidden_ability = clear_ability[4]
elif len(clear_ability) < 5:
    ability = clear_ability[0]
    hidden_ability = clear_ability[2]
print(len(clear_ability), ability, hidden_ability)
clear_ability

6 ('Cute Charm', 'Competitive') Frisk


['Cute Charm', 'Cute Charm', 'Competitive', 'Competitive', 'Frisk', 'Frisk']